In [64]:
import sys
sys.path.insert(0, '../gofher')

import os
import matplotlib.image as mpimg
import numpy as np

from gofher import run_gofher
from visualize import visualize
from file_helper import write_csv,check_if_folder_exists_and_create
from spin_parity import read_spin_parity_galaxies_label_from_csv, standardize_galaxy_name

In [65]:
survery_to_use = "sdss"

BANDS_IN_ORDER = ['g','r','i','z'] #Important: Must stay in order of BLUEST to REDDEST Waveband (Editting this will cause gofher to no longer correctly evaluate redder side of galaxy)
REF_BANDS_IN_ORDER = ['r','i','z','g'] #The prefernce each waveband being choosen as refernce band from highest priority to lowest priority

In [66]:
figure_to_run_on = "figure9"

In [67]:
generate_verbose_csv = False
generate_ebm_csv = True
generate_visualization = False
save_visualization = False

In [68]:
#Important: Make sure you update these values:
path_to_catalog_data = "..\\..\\spin-parity-catalog-data"
path_to_output = "..\\..\\gofher-data\\{}\\source_extraction".format(survery_to_use)

In [69]:
def get_fits_path(name,band):
    """the file path of where existing fits files can be found"""
    return os.path.join(path_to_catalog_data,survery_to_use,figure_to_run_on,name,"{}_{}.fits".format(name,band))

def get_color_image_path(name):
    return os.path.join(path_to_catalog_data,survery_to_use,figure_to_run_on,name,"{}_color.png".format(name))

def get_path_to_catalog_csv():
    return os.path.join(path_to_catalog_data,"catalog","{}.csv".format(figure_to_run_on))

In [70]:
def get_paper_dark_side_labels():
    return read_spin_parity_galaxies_label_from_csv(get_path_to_catalog_csv())

def get_galaxies():
    return os.listdir(os.path.join(path_to_catalog_data,survery_to_use,figure_to_run_on))

In [71]:
def run_gofher_on_catalog():
    paper_labels = get_paper_dark_side_labels()

    verbose_header = []
    verbose_rows = []

    ebm_header = []
    ebm_rows = []

    i = 1


    for name in get_galaxies():

        if standardize_galaxy_name(name) not in paper_labels:
            print("skippimg",name)
            continue

        print(name, i,"of",len(get_galaxies()))

        try:
            paper_label = paper_labels[standardize_galaxy_name(name)]
            gal = run_gofher(name,get_fits_path,BANDS_IN_ORDER,REF_BANDS_IN_ORDER, paper_label)

            if generate_verbose_csv:
                (header,row) = gal.get_verbose_csv_header_and_row(BANDS_IN_ORDER,paper_label)
                if len(verbose_header) == 0: verbose_header = header
                verbose_rows.append(row)

            if generate_ebm_csv:
                (header,row) = gal.get_ebm_csv_header_and_row(BANDS_IN_ORDER, paper_label)
                if len(ebm_header) == 0: ebm_header = header
                ebm_rows.append(row)

            if generate_visualization:
                save_path = ''
                
                if save_visualization:
                    sub_folder = os.path.join(path_to_output,figure_to_run_on)
                    check_if_folder_exists_and_create(sub_folder)
                    save_path = os.path.join(sub_folder,"{}.png".format(name))

                color_image = mpimg.imread(get_color_image_path(name))
                color_image = np.flipud(color_image) #flip the image left to right to fix orientation
                visualize(gal,color_image,BANDS_IN_ORDER,paper_label,save_path=save_path)
        except Exception as e:
            print(e)
        i += 1

    if generate_verbose_csv:
        verbose_csv_path = os.path.join(path_to_output,"{}_verbose.csv".format(figure_to_run_on))
        write_csv(verbose_csv_path,verbose_header,verbose_rows)

    if generate_ebm_csv:
        ebm_csv_path = os.path.join(path_to_output,"{}_ebm.csv".format(figure_to_run_on))
        write_csv(ebm_csv_path,ebm_header,ebm_rows)

In [72]:
if not os.path.exists(path_to_catalog_data):
    raise ValueError("The path to the catalog is not found {} - make sure you update path_to_catalog_data".format(path_to_catalog_data))

if not os.path.exists(path_to_output):
    raise ValueError("The path output is not found {} - make sure you update path_to_output".format(path_to_output))


run_gofher_on_catalog()

IC1151 1 of 268
IC1199 2 of 268
IC1256 3 of 268
IC1528 4 of 268
IC2095 5 of 268
IC2487 6 of 268
IC4566 7 of 268
IC5309 8 of 268
IC674 9 of 268
IC776 10 of 268
NGC1 11 of 268
NGC1012 12 of 268
NGC1042 13 of 268
NGC1068 14 of 268
NGC1073 15 of 268
NGC1087 16 of 268
NGC160 17 of 268
NGC1614 18 of 268
NGC1645 19 of 268
NGC1677 20 of 268
NGC171 21 of 268
NGC177 22 of 268
NGC180 23 of 268
NGC192 24 of 268
NGC214 25 of 268
NGC217 26 of 268
NGC23 27 of 268
NGC234 28 of 268
NGC237 29 of 268
NGC2449 30 of 268
NGC2486 31 of 268
NGC2487 32 of 268
NGC2500 33 of 268
NGC2540 34 of 268
NGC2552 35 of 268
NGC2553 36 of 268
NGC257 37 of 268
NGC2604 38 of 268
NGC2608 39 of 268
NGC2730 40 of 268
NGC2776 41 of 268
NGC2805 42 of 268
NGC2906 43 of 268
NGC2916 44 of 268
NGC2964 45 of 268
NGC2998 46 of 268
NGC3057 47 of 268
NGC3106 48 of 268
NGC3166 49 of 268
NGC3206 50 of 268
NGC3319 51 of 268
NGC3344 52 of 268
NGC3346 53 of 268
NGC3351 54 of 268
NGC3359 55 of 268
NGC3367 56 of 268
NGC3381 57 of 268
NGC3395 58